### Raw Data Processing

In [32]:
# import libraries

import os
import io
import re
import csv
from string import punctuation
from time import time
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize

In [33]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/lea/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

#### Helper Functions

In [51]:
def process_ann(ann_file, test=False):
    """Helper function that reads a .ann file,
       strips out newline characters, splits the tab-delimited entries,
       and extracts information for identifying entities and relations 
       in corresponding .txt file.
       Also adds negative relations within the span of +/- 5 entities 
       from each trigger word.
       If generating test data, only includes positive relations within
       the span of +/- 5 entities from the trigger word.
       
       Input:
       ann_file = tab-delimited brat annotation file with the following format
                  NER: [entity_ID]\t[label start_offset end_offset]\t[entity]
                  RE:  [relation_ID]\t[relation_type argument1 argument2]
       span = number of entities from the trigger word to look for pairs
       test = whether or not test data is being generated
       
       Outputs:
       cleaned_offsets = list of tuples for labeling corresponding .txt file
                         format: (offset, label, entity ID)
       relations = list of tuples for extracting relations from corresponding .txt file
                   format: (relation ID, relation_type, entity ID #1, entity ID #2)
       positives_missed = count of positive relations missed"""
    
    # dictionary to convert NER labels to NER-specific markers 
    # for BERT input coded with seldom used tokens
    ner_markers = {'STARTING_MATERIAL': 'Α', 'REAGENT_CATALYST': 'Β', 'REACTION_PRODUCT': 'Π', 
                   'SOLVENT': 'Σ', 'OTHER_COMPOUND': 'Ο', 'EXAMPLE_LABEL': 'Χ', 
                   'TIME': 'Τ', 'TEMPERATURE': 'Θ', 'YIELD_PERCENT': 'Ψ', 'YIELD_OTHER': 'Υ',
                   'WORKUP': 'Λ', 'REACTION_STEP': 'Δ'}
    
    with io.open(ann_file, 'r', encoding='utf-8', errors='ignore') as f:
        text = [x.strip().split('\t') for x in f.readlines()]
        
    ann = [x for x in text if x[0][0] == 'T']
    rel = [x for x in text if x[0][0] == 'R']
    
    # extract information for identifying entities
    offsets = []
    
    for x in ann:
        entity_id = x[0]
        start = int(x[1].split()[1])
        end = int(x[1].split()[2])
        label = x[1].split()[0]
        
        offsets.append((start, 'S', ner_markers[label], entity_id))
        offsets.append((end, 'E', ner_markers[label], entity_id))
    
    # sort offsets and clean overlapping entries
    sorted_offsets = sorted(offsets, key=lambda x:x[0])
    
    cleaned_offsets = []
    corrections = {}
    
    hold = None
    indicator = None
    
    for tup in sorted_offsets:
        
        if indicator == 'S':
            if tup[1] == 'E':
                cleaned_offsets.append(hold)
                hold = (tup[0], 'O', 'X')
                indicator = tup[1]
            elif tup[1] == 'S':
                corrections.update({tup[3]:hold[2]})
                indicator = '*'
        
        elif indicator == 'E':
            cleaned_offsets.append(hold)
            hold = (tup[0], tup[2], tup[3])
            indicator = tup[1]
        
        elif indicator == '*':
            indicator = 'S'

        else:
            hold = (tup[0], tup[2], tup[3])
            indicator = tup[1]
            
    cleaned_offsets.append(hold)
    
    # extract information for identifying relations
    relations = []
    positives = {}
    included = []
    
    # add positive relations
    for r in rel:
        relation_id = r[0]
        relation_type = r[1].split()[0]
        entity1 = r[1].split()[1][5:]
        entity2 = r[1].split()[2][5:]
        
        if entity1 in corrections.keys():
            entity1 = corrections[entity1]
        if entity2 in corrections.keys():
            entity2 = corrections[entity2]
        
        positives.update({(entity1, entity2):(relation_id, relation_type)})
        
        if not test:
            relations.append((relation_id, relation_type, entity1, entity2))
            included.append((entity1, entity2))
    
    # negative relations
    negatives = []
    triggers = {x[2] for x in cleaned_offsets 
                if (x[1] == ner_markers['WORKUP'] or x[1] == ner_markers['REACTION_STEP'])}
    entity_order = [x[2] for x in cleaned_offsets if (x[2] != 'X' and x[1] != ner_markers['EXAMPLE_LABEL'])]
    trigger_indices = {i for i in range(len(entity_order)) if entity_order[i] in triggers}
    
    # find negative relations
    for index in trigger_indices:
        
        # find indices in span of +/- 5 from trigger
        find_span = [index - (i+1) for i in range(5)] + [index + (i+1) for i in range(5)]
        real_span = [i for i in find_span if (i >= 0 and i < len(entity_order))]
        final_span = [i for i in real_span if i not in trigger_indices]
        
        # make tuples of trigger words and entities from span indices
        potential_pairs = [(entity_order[index], entity_order[i]) for i in final_span]
        
        # check if tuple is in positives
        # only add to negatives if not in positives
        for pair in potential_pairs:
            if pair in positives.keys():
                if not test:
                    continue
                elif test:
                    if pair not in included:
                        relations.append((positives[pair][0], positives[pair][1], pair[0], pair[1]))
                        included.append(pair)
            else:
                negatives.append(pair)
    
    # make a list of negative relations 
    # in same format as positives to add to relations list
    i = 0
    for pair in negatives:
        negative_id = f'N{i}'
        relations.append((negative_id, 'NONE', pair[0], pair[1]))
        i += 1
        
    # count how many positive relations were missed
    positives_missed = len(positives.keys()) - len(included) 
    
    return cleaned_offsets, relations, positives_missed

In [35]:
def ann_chunker(txt_file, offsets):
    """Helper function that reads in a .txt file as one string,
       divides it based on the cleaned offsets from its .ann file
       and labels chunks with NER tags
       
       Inputs:
       txt_file = file that contains all the patent text
                  considered as one sentence in this task
       offsets = list of tuples for labeling corresponding .txt file
                 format: (offset, label, entity ID)
       
       Output:
       ann_chunks = list of annotated chunks based on .ann file offsets
                    format: (chunk, label, entity ID)"""
    
    with io.open(txt_file, 'r', encoding='utf-8', errors='ignore') as text:
        full_text = text.read()
    
    start = 0
    end = offsets[0][0]
    label = 'O'
    entity_id = 'X'
    
    ann_chunks = [(full_text[:end], label, entity_id)]
    
    for i in range(len(offsets)):
        start = offsets[i][0]
        label = offsets[i][1]
        entity_id = offsets[i][2]
        
        if i < len(offsets) - 1:
            end = offsets[i+1][0]
            term = [(full_text[start:end], label, entity_id)]
            if term[0]:
                ann_chunks.extend(term)
        
        else:
            term = [(full_text[start:], label, entity_id)]  
            ann_chunks.extend(term)
    
    return ann_chunks

In [36]:
def relation_input(snippet_id, rel_tup, chunks):
    """Helper function that creates one input snippet for BERT SRE:
       Inserts entity markers and truncates snippet to include only
       sentences containing the entities
    
       Inputs:
       snippet_id = filename of snippet
       rel_tup = tuple from relations list generated by process_ann()
             format: (relation ID, relation_type, entity ID #1, entity ID #2)
       chunks = list of annotated chunks from ann_chunker()
    
       Output:
       rel_input = input snippet ready for BERT SRE
                   format: [snippet_id+relation_id]/t[relation_type]/t[cleaned snippet with ner markers]"""
    
    # unpack relation_tup
    relation_id = rel_tup[0]
    relation_type = rel_tup[1]
    entity_list = [rel_tup[2], rel_tup[3]]
    
    new_id = snippet_id + '-' + relation_id
    
    # build cleaned snippet with ner markers
    snippets = []
    i = 1
    
    for tup in chunks:
        chunk, label, entity = tup
        
        # clean chunk: remove punctuation,
        # word tokenize if not an entity
        # split by whitespace if entity
        processed_chunk = []
            
        if label == 'O':
            nopunct = re.sub(r'[,/()":\-\[\]\']', '', chunk.strip())
            sentences = sent_tokenize(nopunct)
            if sentences:
                for s in sentences:
                    for x in word_tokenize(s):
                        processed_chunk.append(x)
                
        else:
            nopunct = re.sub(r'[,/()":\-\[\]\']', '', chunk)
            tokens = [x for x in nopunct.split(' ') if x]
            for t in tokens:
                processed_chunk.append(t)
        
        # add ner markers before and after entities in relation
        if entity in entity_list:
            snippets.append(label)
            if i == 1:
                e1 = label
            snippets.extend(processed_chunk)
            snippets.append(f'[/E{i}]')
            i += 1
        
        else:
            snippets.extend(processed_chunk)

    # keep only sentences containing the entities
    # sentences are marked by periods
    e1_index = snippets.index(e1)
    e2_index = snippets.index('[/E2]')
    
    periods = [i for i in range(len(snippets)) if snippets[i] == '.']
    periods_before = [i for i in periods if i < e1_index]
    periods_after = [i for i in periods if i > e2_index]
    
    if periods_before:
        start = max(periods_before) + 1
    else:
        start = 0
    
    if periods_after:
        end = min(periods_after) + 1
    else:
        end = None
    
    truncated_snippet = snippets[start:end]
    
    # join snippet chunks to one clean snippet
    cleaned_snippet = ' '.join(truncated_snippet)
    
    return [new_id, relation_type, cleaned_snippet]

In [37]:
def generate_re_files(filepaths, output_path, test=False):
    """Helper function that reads .txt and corresponding .ann files from a path
       and generates csv file with snippets ready for BERT SRE (one snippet per line)
       
       Inputs:
       filepaths = filepaths (folder + filename, but no extension) for .txt and .ann files
       output_path = filepath (folder + filename, but no extension) for output file
       missed_count = count of total positive relations missed"""
    
    start = time()
    
    snippets = []
    missed_positives = []
    
    for file in filepaths:
        
        snippet_id = file[-4:]
        
        cleaned_offsets, relations, missed = process_ann(f'{file}.ann', test=test)
        missed_positives.append(missed)
        chunks = ann_chunker(f'{file}.txt', cleaned_offsets)
        
        for tup in relations:
            line = relation_input(snippet_id, tup, chunks)
            snippets.append(line)
    
    missed_count = sum(missed_positives)
    print(f'Number of positive relations missed: {missed_count}')
    
    with open(f'{output_path}.csv', 'w') as f:
        writer = csv.writer(f, delimiter='\t')
        writer.writerows(snippets)
    
    end = time() - start
    print(f'Finished in {end:.3f} seconds')

In [38]:
def generate_re_files_subsampled(filepaths, output_path, fraction=0.5):
    """Helper function that reads .txt and corresponding .ann files from a path
       and generates csv file with snippets ready for BERT SRE (one snippet per line).
       Subsampled 25% of negative relations randomly.
       
       Inputs:
       filepaths = filepaths (folder + filename, but no extension) for .txt and .ann files
       output_path = filepath (folder + filename, but no extension) for output file"""
    
    start = time()
    
    snippets = []
    missed_positives = []
    
    for file in filepaths:
        
        snippet_id = file[-4:]
        
        cleaned_offsets, relations, missed = process_ann(f'{file}.ann')
        missed_positives.append(missed)
        chunks = ann_chunker(f'{file}.txt', cleaned_offsets)
        
        # add all positive relations
        positive_relations = [tup for tup in relations if tup[0][0] == 'R']
        
        for tup in positive_relations:
            line = relation_input(snippet_id, tup, chunks)
            snippets.append(line)
        
        # keep only 25% of negative relations
        negative_relations = [tup for tup in relations if tup[0][0] == 'N']
        neg_count = len(negative_relations)
        np.random.seed(424)
        negative_keep = np.random.binomial(1, fraction, neg_count)
        
        for i in range(neg_count):
            if negative_keep[i] == 1:
                line = relation_input(snippet_id, negative_relations[i], chunks)
                snippets.append(line)
                
    missed_count = sum(missed_positives)
    print(f'Number of positive relations missed: {missed_count}')
    
    with open(f'{output_path}.csv', 'w') as f:
        writer = csv.writer(f, delimiter='\t')
        writer.writerows(snippets)
    
    end = time() - start
    print(f'Finished in {end:.3f} seconds')

#### Test the Helper Functions

In [39]:
test_path = '../raw_data/EE/ee_train/1069'

In [40]:
with io.open(f'{test_path}.txt', 'r', encoding='utf-8', errors='ignore') as text:
    full_text = text.read()
full_text

"Intermediate 2: 2,3,4,5,6,7-hexahydropyrrolo[3',4':3,4]pyrazolo[1 ,5-b][1,2]thiazine-1,1-dioxide p-toluenesulfonate\nStep 1: tert-butyl 3-iodo-4,6-dihydropyrrolo[3,4-c]pyrazole-5(2H)-carboxylate\nTo a solution of tert-butyl 4,6-dihydropyrrolo[3,4-c]pyrazole-5(2H)-carboxylate (9.41 g, 45 mmol) in 200 mL 1,2-dichloroethane was added N-iodosuccinimide (13.16 g, 58.5 mmol), and refluxed overnight. The solvent was removed by evaporation, and the residue was purified by silica gel column chromatography to give tert-butyl 3-iodo-4,6-dihydropyrrolo[3,4-c]pyrazole-5(2H)-carboxylate (4.66 g). Yield: 31%."

In [41]:
with io.open(f'{test_path}.ann', 'r', encoding='utf-8', errors='ignore') as text:
    ann = [x.strip().split('\t') for x in text.readlines()] #if x.strip().split('\t')[0][0] == 'R']
ann

[['T12', 'WORKUP 455 463', 'purified'],
 ['T0', 'EXAMPLE_LABEL 121 122', '1'],
 ['T13', 'REACTION_STEP 375 383', 'refluxed'],
 ['T1', 'TEMPERATURE 375 383', 'refluxed'],
 ['T2', 'YIELD_PERCENT 595 598', '31%'],
 ['T14', 'REACTION_STEP 325 330', 'added'],
 ['T3',
  'REACTION_PRODUCT 508 577',
  'tert-butyl 3-iodo-4,6-dihydropyrrolo[3,4-c]pyrazole-5(2H)-carboxylate'],
 ['T4',
  'REACTION_PRODUCT 124 193',
  'tert-butyl 3-iodo-4,6-dihydropyrrolo[3,4-c]pyrazole-5(2H)-carboxylate'],
 ['T5', 'REAGENT_CATALYST 331 348', 'N-iodosuccinimide'],
 ['T15', 'REACTION_STEP 503 507', 'give'],
 ['T6', 'EXAMPLE_LABEL 13 14', '2'],
 ['T7', 'OTHER_COMPOUND 467 477', 'silica gel'],
 ['T8',
  'STARTING_MATERIAL 211 273',
  'tert-butyl 4,6-dihydropyrrolo[3,4-c]pyrazole-5(2H)-carboxylate'],
 ['T9', 'SOLVENT 302 320', '1,2-dichloroethane'],
 ['T10', 'YIELD_OTHER 579 585', '4.66 g'],
 ['T11',
  'OTHER_COMPOUND 16 115',
  "2,3,4,5,6,7-hexahydropyrrolo[3',4':3,4]pyrazolo[1 ,5-b][1,2]thiazine-1,1-dioxide p-toluene

In [52]:
test_offsets, test_relations, test_missed = process_ann(f'{test_path}.ann', test=False)

corrections: {'T1': 'T13'}
Trigger indices: {9, 4, 6, 7}
find span: [8, 7, 6, 5, 4, 10, 11, 12, 13, 14]
real span: [8, 7, 6, 5, 4, 10, 11, 12]
final span: [8, 5, 10, 11, 12]
potential pairs: [('T15', 'T7'), ('T15', 'T5'), ('T15', 'T3'), ('T15', 'T10'), ('T15', 'T2')]
find span: [3, 2, 1, 0, -1, 5, 6, 7, 8, 9]
real span: [3, 2, 1, 0, 5, 6, 7, 8, 9]
final span: [3, 2, 1, 0, 5, 8]
potential pairs: [('T14', 'T9'), ('T14', 'T8'), ('T14', 'T4'), ('T14', 'T11'), ('T14', 'T5'), ('T14', 'T7')]
find span: [5, 4, 3, 2, 1, 7, 8, 9, 10, 11]
real span: [5, 4, 3, 2, 1, 7, 8, 9, 10, 11]
final span: [5, 3, 2, 1, 8, 10, 11]
potential pairs: [('T13', 'T5'), ('T13', 'T9'), ('T13', 'T8'), ('T13', 'T4'), ('T13', 'T7'), ('T13', 'T3'), ('T13', 'T10')]
find span: [6, 5, 4, 3, 2, 8, 9, 10, 11, 12]
real span: [6, 5, 4, 3, 2, 8, 9, 10, 11, 12]
final span: [5, 3, 2, 8, 10, 11, 12]
potential pairs: [('T12', 'T5'), ('T12', 'T9'), ('T12', 'T8'), ('T12', 'T7'), ('T12', 'T3'), ('T12', 'T10'), ('T12', 'T2')]
negative pa

In [43]:
test_missed

0

In [44]:
test_offsets

[(13, 'Χ', 'T6'),
 (14, 'O', 'X'),
 (16, 'Ο', 'T11'),
 (115, 'O', 'X'),
 (121, 'Χ', 'T0'),
 (122, 'O', 'X'),
 (124, 'Π', 'T4'),
 (193, 'O', 'X'),
 (211, 'Α', 'T8'),
 (273, 'O', 'X'),
 (302, 'Σ', 'T9'),
 (320, 'O', 'X'),
 (325, 'Δ', 'T14'),
 (330, 'O', 'X'),
 (331, 'Β', 'T5'),
 (348, 'O', 'X'),
 (375, 'Δ', 'T13'),
 (383, 'O', 'X'),
 (455, 'Λ', 'T12'),
 (463, 'O', 'X'),
 (467, 'Ο', 'T7'),
 (477, 'O', 'X'),
 (503, 'Δ', 'T15'),
 (507, 'O', 'X'),
 (508, 'Π', 'T3'),
 (577, 'O', 'X'),
 (579, 'Υ', 'T10'),
 (585, 'O', 'X'),
 (595, 'Ψ', 'T2'),
 (598, 'O', 'X')]

In [45]:
test_relations

[('R0', 'ARG1', 'T12', 'T7'),
 ('R1', 'ARG1', 'T14', 'T9'),
 ('R2', 'ARG1', 'T14', 'T5'),
 ('R3', 'ARG1', 'T14', 'T8'),
 ('R4', 'ARGM', 'T14', 'T13'),
 ('R5', 'ARG1', 'T15', 'T3'),
 ('R6', 'ARGM', 'T15', 'T10'),
 ('R7', 'ARGM', 'T15', 'T2'),
 ('N0', 'NONE', 'T15', 'T7'),
 ('N1', 'NONE', 'T15', 'T5'),
 ('N2', 'NONE', 'T14', 'T4'),
 ('N3', 'NONE', 'T14', 'T11'),
 ('N4', 'NONE', 'T14', 'T7'),
 ('N5', 'NONE', 'T13', 'T5'),
 ('N6', 'NONE', 'T13', 'T9'),
 ('N7', 'NONE', 'T13', 'T8'),
 ('N8', 'NONE', 'T13', 'T4'),
 ('N9', 'NONE', 'T13', 'T7'),
 ('N10', 'NONE', 'T13', 'T3'),
 ('N11', 'NONE', 'T13', 'T10'),
 ('N12', 'NONE', 'T12', 'T5'),
 ('N13', 'NONE', 'T12', 'T9'),
 ('N14', 'NONE', 'T12', 'T8'),
 ('N15', 'NONE', 'T12', 'T3'),
 ('N16', 'NONE', 'T12', 'T10'),
 ('N17', 'NONE', 'T12', 'T2')]

In [46]:
trial_sentence = ann_chunker(f'{test_path}.txt', test_offsets)

In [18]:
trial_sentence[:3]

[('Example ', 'O', 'X'), ('19', 'Χ', 'T8'), ('\nPreparation of ', 'O', 'X')]

In [19]:
trial_snippet = relation_input('0000', test_relations[0], trial_sentence)
trial_snippet

['0000-R4',
 'ARG1',
 'Example 19 Preparation of N46S8R722difluoropropyl8methyl6789tetrahydro3Hpyrazolo43fisoquinolin6yl3methoxyphenyl13fluoropropylazetidin3amine DMF 2 mL and Β DIPEA [/E1] 0.074 mL 0.42 mmol were Δ added [/E2] sequentially to a flask charged with N46S8R722difluoropropyl8methyl6789tetrahydro3Hpyrazolo43fisoquinolin6yl3methoxyphenylazetidin3amine 75 mg 0.17 mmol .']

In [30]:
generate_re_files([test_path], '../raw_data/test', test=True)
with io.open('../raw_data/test.csv', 'r', encoding='utf-8', errors='ignore') as sample:
    output = sample.readlines()
len(output)

Number of positive relations missed: 0
Finished in 0.214 seconds


63

In [21]:
generate_re_files_subsampled([test_path], '../raw_data/test2')
with io.open('../raw_data/test2.csv', 'r', encoding='utf-8', errors='ignore') as sample:
    output2 = sample.readlines()
len(output2)

Number of positive relations missed: 0
Finished in 0.145 seconds


37

#### Process the Raw Data

In [22]:
# generate sample set
path_sample = '../raw_data/sample_ee'
filenames_sample = list({x[:4] for x in os.listdir(path_sample) if x[0] != '.'})
filepath_sample = [f'{path_sample}/{x}' for x in filenames_sample]

output_sample = '../data/sre_ner/sre_ner_sample'
generate_re_files(filepath_sample, output_sample)

with io.open(f'{output_sample}.csv', 'r', encoding='utf-8', errors='ignore') as sample:
    output = sample.readlines()
check = [1 for x in output]
print(f'Number of sample snippets: {len(check)}')

Number of positive relations missed: 0
Finished in 5.075 seconds
Number of sample snippets: 1977


In [23]:
# generate filename list for train, dev, and test sets
path_train = '../raw_data/EE/ee_train'
filenames_train = list({x[:4] for x in os.listdir(path_train) if x[0] != '.'})
print(f'Number of train files: {len(filenames_train)}')

path_dev = '../raw_data/EE/ee_dev'
filenames_dev = list({x[:4] for x in os.listdir(path_dev) if x[0] != '.'})
print(f'Number of dev files: {len(filenames_dev)}')

path_test = '../raw_data/EE/ee_test'
filenames_test = list({x[:4] for x in os.listdir(path_test) if x[0] != '.'})
print(f'Number of test files: {len(filenames_test)}')

path_test_ann = '../raw_data/EE/ee_test_ann'
filenames_test_ann = list({x[:4] for x in os.listdir(path_test_ann) if x[0] != '.'})
print(f'Number of test .ann files: {len(filenames_test_ann)}')

Number of train files: 900
Number of dev files: 225
Number of test files: 9999
Number of test .ann files: 375


In [24]:
# check how many test .txt files match the .ann files
intersect = list(set(filenames_test) & set(filenames_test_ann))
len(intersect)

375

In [25]:
# generate train set
filepath_train = [f'{path_train}/{x}' for x in filenames_train]

output_train = '../data/sre_ner/sre_ner_train'
generate_re_files(filepath_train, output_train)

with io.open(f'{output_train}.csv', 'r', encoding='utf-8', errors='ignore') as sample:
    output = sample.readlines()
check = [1 for x in output]
print(f'Number of train snippets: {len(check)}')

Number of positive relations missed: -16
Finished in 181.820 seconds
Number of train snippets: 45821


In [26]:
# generate dev set
filepath_dev = [f'{path_dev}/{x}' for x in filenames_dev]

output_dev = '../data/sre_ner/sre_ner_dev'
generate_re_files(filepath_dev, output_dev)

with io.open(f'{output_dev}.csv', 'r', encoding='utf-8', errors='ignore') as sample:
    output = sample.readlines()
check = [1 for x in output]
print(f'Number of dev sentences: {len(check)}')

Number of positive relations missed: 0
Finished in 34.313 seconds
Number of dev sentences: 10673


In [31]:
# generate test set
filepath_test = [f'{path_test}/{x}' for x in intersect]

output_test = '../data/sre_ner/sre_ner_test'
generate_re_files(filepath_test, output_test, test=True)

with io.open(f'{output_test}.csv', 'r', encoding='utf-8', errors='ignore') as sample:
    output = sample.readlines()
check = [1 for x in output if x[:8]]
print(f'Number of test sentences: {len(check)}')

Number of positive relations missed: 25
Finished in 64.315 seconds
Number of test sentences: 18488


#### Subsampled data

In [28]:
# generate train set
filepath_train = [f'{path_train}/{x}' for x in filenames_train]

output_train = '../data/sre_ner/sre_ner_train_subsampled'
generate_re_files_subsampled(filepath_train, output_train)

with io.open(f'{output_train}.csv', 'r', encoding='utf-8', errors='ignore') as sample:
    output = sample.readlines()
check = [1 for x in output]
print(f'Number of train snippets: {len(check)}')

Number of positive relations missed: -16
Finished in 114.969 seconds
Number of train snippets: 27791


In [29]:
# generate dev set
filepath_dev = [f'{path_dev}/{x}' for x in filenames_dev]

output_dev = '../data/sre_ner/sre_ner_dev_subsampled'
generate_re_files_subsampled(filepath_dev, output_dev)

with io.open(f'{output_dev}.csv', 'r', encoding='utf-8', errors='ignore') as sample:
    output = sample.readlines()
check = [1 for x in output]
print(f'Number of dev sentences: {len(check)}')

Number of positive relations missed: 0
Finished in 21.238 seconds
Number of dev sentences: 6447
